# Carga de Datos

In [ ]:
!wget https://www.dropbox.com/s/8oggitmn7vlhv4v/project.zip?dl=0s
!unzip -qq project.zip
!ls project

# Librerías y Funciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import find_contours
from tqdm.auto import tqdm

In [ ]:
def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_img,echo='off'):
  # X = ImageLoad('project/images/X',i)        # carga imagen Rx i en X
  # Y = ImageLoad('project/segmentation/Y',i)  # carga segmentacion i en Y
  st   = prefix + num2fixstr(num_img,4) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = 255*plt.imread(st)
  return img

def EdgeView(X,Ys,Y=[]):
  plt.figure(figsize=(15,15))
  plt.imshow(X,cmap='gray')

  cc = 'r'

  if len(Y)!=0:
    contours = find_contours(np.float32(Y), 0.5)
    for n, contour in enumerate(contours):
      plt.plot(contour[:, 1], contour[:, 0], color=cc, linewidth=3)
    cc = 'y'


  contours = find_contours(np.float32(Ys), 0.5)
  for n, contour in enumerate(contours):
    plt.plot(contour[:, 1], contour[:, 0], color=cc, linewidth=3)


  


# Carga de imagen i

In [ ]:
i = 10
# X-ray image
X = ImageLoad('project/images/X',i)
plt.figure(figsize=(15,15))
plt.imshow(X,cmap='gray')
plt.title('Rx X'+num2fixstr(i,4)+'.png')
plt.show()

# Ideal Segmentation (Ground Truth)
Y = ImageLoad('project/segmentation/Y',i)
plt.figure(figsize=(15,15))
plt.imshow(Y,cmap='gray')
plt.title('GT Y'+num2fixstr(i,4)+'.png')
plt.show()


# X-ray and ideal Segmentation
EdgeView(X,Y)
plt.title('Rx and GT '+num2fixstr(i,4))
plt.show()

# Segmentación

In [ ]:
def LungSegmentation(X):
  Ys = 255.*(X>100)
  return Ys


In [ ]:
i = 10
# X-ray image
X = ImageLoad('project/images/X',i)

# Ideal Segmentation (Ground Truth)
Y = ImageLoad('project/segmentation/Y',i)

# X-ray + real and ideal Segmentation
Ys = LungSegmentation(X)
EdgeView(X,Ys,Y)
plt.title('Segmentation of X'+num2fixstr(i,4)+'.png (red: ideal, yellow: real)')
plt.show()


# Métricas estadísticas

## Funciones

In [ ]:
def Detections(Y,Ys):
  GT = Y>128
  D  = Ys>128
  GTn = np.logical_not(GT)
  Dn  = np.logical_not(D)
  TP = np.sum(np.logical_and(GT ,D ))
  TN = np.sum(np.logical_and(GTn,Dn))
  FP = np.sum(np.logical_and(GTn,D ))
  FN = np.sum(np.logical_and(GT ,Dn ))
  return TP,TN,FP,FN

def RatesF1(TP,TN,FP,FN):
  TPR = TP/(TP+FN)
  FPR = FP/(FP+TN)
  F1 = 2*TP / (2*TP + FP + FN)
  return TPR,FPR,F1


def statisticsImage(segFunction,i):
  X  = ImageLoad('project/images/X',i)        # X-ray image
  Y  = ImageLoad('project/segmentation/Y',i)  # Ideal segmentation (ground truth)
  Ys = segFunction(X)                    # Real Segmentation
  (TP,TN,FP,FN) = Detections(Y,Ys)
  (TPR,FPR,F1)  = RatesF1(TP,TN,FP,FN)
  return TPR,FPR,F1

def statisticsAllImages(segFunction):
  (TP,TN,FP,FN) = (0,0,0,0)
  for k in tqdm(range(40)):
    i = k+1
    X = ImageLoad('project/images/X',i)
    Y = ImageLoad('project/segmentation/Y',i)
    Ys = LungSegmentation(X)
    (TPi,TNi,FPi,FNi) = Detections(Y,Ys)
    TP = TP+TPi
    TN = TN+TNi
    FP = FP+FPi
    FN = FN+FNi
  (TPR,FPR,F1)  = RatesF1(TP,TN,FP,FN)
  return TPR,FPR,F1

## Estadísticas para imagen i

In [ ]:
(TPR,FPR,F1)  = statisticsImage(LungSegmentation,10)
print("TPR = {:.4f}".format(TPR))
print("FPR = {:.4f}".format(FPR))
print("F1  = {:.4f}".format(F1))

## Estadísticas para todas las imágenes


In [ ]:
(TPR,FPR,F1) = statisticsAllImages(LungSegmentation)
print("TPR = {:.4f}".format(TPR))
print("FPR = {:.4f}".format(FPR))
print("F1  = {:.4f}".format(F1))